In [4]:
from lit_classes import Bea2019Data, GectorBertModel, GECE_ERROR_TYPES
from attention_analysis import attention_analysis
# reuse the LIT classes as they're already packaged for analysis
from warnings import simplefilter

simplefilter(action='ignore', category=DeprecationWarning)

model = GectorBertModel('bert_0_gector.th')
data = Bea2019Data('data/test.jsonl', gece_tags=True)
pearson, regression, argmax = attention_analysis(model, data, model.ATTENTION_HEADS, model.ATTENTION_LAYERS)

processing batches:   0%|          | 0/2 [00:00<?, ?it/s]

Inference time 2.0334646701812744
Iteration 1. Predicted 100.0% of sentences.
Inference time 1.157778024673462
Iteration 1. Predicted 100.0% of sentences.


processing results: 0it [00:00, ?it/s]

In [ ]:
pearson_lists = {ex: [] for ex in GECE_ERROR_TYPES}
regression_list = {ex: [] for ex in GECE_ERROR_TYPES}
argmax_list = {ex: [] for ex in GECE_ERROR_TYPES}
